<a href="https://colab.research.google.com/github/THANTANIN/long/blob/main/3c_BERT_finetuning_for_token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## State Condition

In [8]:
target = "status"
model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 16
num_epoch = 1
condition = f"pretokenized_conn_status"
limit_n_tokens = 200
save="no"
#save=("no","epoch", or "step")

In [9]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


## Set Environment

In [3]:
!pip install datasets transformers seqeval
!pip install sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5c122b8ddf72ed13dedf34317a8f1df2a4bf035d08a2b2cff9081cba67a6c657
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resol

In [4]:
! pip install evaluate

from evaluate import load


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import re

from transformers import (
    AutoModel, AutoTokenizer, AutoConfig,
    AutoModelForTokenClassification,
    TrainingArguments, Trainer,
    DataCollatorForTokenClassification
)

from datasets import (
    load_dataset,
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)
from sklearn.metrics import classification_report

In [6]:
lm_name = 'airesearch/wangchanberta-base-att-spm-uncased'
model = AutoModel.from_pretrained(lm_name)
tokenizer = AutoTokenizer.from_pretrained(lm_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

## Loading the dataset

In [7]:
def get_df(filepath,num_sent=1):
    """
    get dataframe from train/test/dev folder of .txt, /t segmented raw text

    Parameters
    ----------
    filepath : string
    num_sent : int
        number of sentences for each row of dataframe, default at 1 sentent per row

    Returns
    -------
    dataframe
    """
    keyList = ['tokens','conn_label']
    data = {key: [] for key in keyList}
    for filename in glob.glob(os.path.join(filepath, '*.txt')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:

            column1 = []
            column2 = []
            count_sent = 0
            for i,line in enumerate(f.readlines()):
                entry = line.rstrip('\n').split('\t')

                if(len(entry)!=7):
                    count_sent+=1

                    entry = ['<_>','space','space','space','space','SP','-1']
                    if(count_sent ==  num_sent):
                        data["tokens"].append(column1)
                        data["conn_label"].append(column2)
                        column1 = []
                        column2 = []
                        count_sent = 0
                        continue

                column1.append("".join(str(entry[0]).split()))#remove internal space
                if((entry[5]=='N')  or  (entry[5]=='SP')):
                    column2.append(str(entry[5]))
                else:
                    if(target=='sense'):
                        column2.append(str(entry[5]))
                    elif(target=='sense_lv1'):
                        sense = entry[5].split('.')
                        column2.append(sense[0])
                    elif(target=='sense_lv2'):
                        #connective sense_lv2
                        sense = entry[5].split('.')
                        column2.append(sense[0]+"."+sense[1])
                    else:
                        #connective status
                        column2.append('CON')
    df=pd.DataFrame.from_dict(data)
    df['n_tokens'] =df.tokens.apply(len)

    return df


In [10]:
#!gdown 1CvFRCHV3jLNdi4nyuuNUlJBiLXjDagBj
#!unzip 'BERT Token Classification Data.zip'
ds= load_dataset("pythainlp/thaisum")



README.md:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

(…)-00000-of-00006-6e20f63d89ca77c0.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00001-of-00006-9989c60d1f9140e4.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00002-of-00006-d228c68dd60b0513.parquet:   0%|          | 0.00/193M [00:00<?, ?B/s]

(…)-00003-of-00006-d0c1675c6a75d466.parquet:   0%|          | 0.00/193M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/358868 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/11000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    valid: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
})

In [ ]:
print((type(ds))

In [9]:
train_df=get_df('BERT Token Classification Data/train/')
dev_df=get_df('BERT Token Classification Data/dev/')
test_df=get_df('BERT Token Classification Data/test/')

In [46]:
print(train_df , test_df.shape , dev_df.shape)
train_df = train_df[train_df.n_tokens<limit_n_tokens]
test_df = test_df[test_df.n_tokens<limit_n_tokens]
dev_df = dev_df[dev_df.n_tokens<limit_n_tokens]
print(train_df.shape , test_df.shape , dev_df.shape)

Empty DataFrame
Columns: [tokens, conn_label, n_tokens]
Index: [] (0, 3) (0, 3)
(0, 3) (0, 3) (0, 3)


เปลี่ยน Dataframe เป็น dictionary เพื่อหลีกเลี่ยง  pyarrow error

In [44]:
train = train_df[["tokens","conn_label"]].to_dict(orient="series")
dev = dev_df[["tokens","conn_label"]].to_dict(orient="series")
test = test_df[["tokens","conn_label"]].to_dict(orient="series")

In [45]:
# prompt: I want to seek item in test = test_df[["tokens","conn_label"]].to_dict(orient="series")

# Assuming you want to find an item based on its index in the 'tokens' list
def seek_item(data, index):
  """Seeks an item in the provided dictionary based on the given index within the 'tokens' list.

  Args:
      data (dict): A dictionary with 'tokens' and 'conn_label' keys,
                    where 'tokens' values are lists.
      index (int): The index to look for in the 'tokens' lists.

  Returns:
      tuple: A tuple containing the token at the specified index and its corresponding
             'conn_label', or None if the index is out of bounds or the data is invalid.
  """
  try:
    tokens = data['tokens']
    conn_labels = data['conn_label']

    if not isinstance(tokens, list) or not isinstance(conn_labels, list):
      print("Error: 'tokens' and 'conn_label' values must be lists.")
      return None

    if not all(isinstance(item, list) for item in tokens):
      print("Error: Items in the 'tokens' list must also be lists.")
      return None

    if 0 <= index < len(tokens):
        return tokens[index], conn_labels[index]
    else:
        print(f"Error: Index {index} out of bounds for the given data.")
        return None

  except KeyError as e:
    print(f"Error: Key '{e}' not found in the provided dictionary.")
    return None
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return None
seek_item(train,100)

Error: 'tokens' and 'conn_label' values must be lists.


กำหนด features ของ Dataset โดยใช้คำสั่ง` Features`
เนื่องจาก ข้อมูลของเราเป็น sequence ของ tokens จึงกำหนดให้


*   column `tokens` เป็น sequence ของ string
*   column `conn_label` เป็น sequence ของ `Classlabel`

ในที่นี้ column `conn_label` จะเป็น sequence ของ Label ['CON','N']
เช่น ` ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'] `หรือ `['N', 'N', 'N', 'N','N', 'N', 'N', 'CON', 'N']`






In [13]:
conn_label = ['CON','N']
conn_features= Features(
                {
                    "tokens": Sequence(Value('string')),
                    "conn_label": Sequence(feature=ClassLabel(names=conn_label))
                })

ใช้คำสั่ง `DatasetDict` เพื่อเก็บข้อมูลเป็น dictionary ของ dataset



In [14]:
data = DatasetDict({'train': Dataset.from_dict(train,features=conn_features),
                 'dev': Dataset.from_dict(dev,features=conn_features),
                 'test': Dataset.from_dict(test,features=conn_features)})

ลองดู features ที่กำหนดไว้

In [15]:
data['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'conn_label': Sequence(feature=ClassLabel(names=['CON', 'N'], id=None), length=-1, id=None)}

## Preprocessing the data

`tokenizer` ของ BERT จะเปลี่ยนข้อมูลจาก list of tokens ให้กลายเป็น input_ids และ attention _mask สำหรับโมเดล
ใช้คำสั่ง `is_split_into_words=True` ถ้าเราใช้ข้อมูลที่ตัดแบ่งคำมาเรียบร้อยแล้ว

In [16]:
tokenizer(['คนร้าย', 'ซุก', 'ระเบิด'],truncation=True, is_split_into_words=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [5, 10, 1343, 10, 6068, 10, 926, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

ตัวอย่างการ tokenized ประโยค `['คนร้าย', 'ซุก', 'ระเบิด', 'ใน', 'รถ', 'จักรยานยนต์', 'ก่อน', 'จุด', 'ชนวน']`

In [17]:
example = data['test']
print(example["tokens"])

[]


In [18]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
print(tokenized_input)

{'input_ids': [5, 6], 'attention_mask': [1, 1]}


สามารถดูได้ว่า BERT ตัดคำใหม่ออกมาเป็นอย่างไร โดยใช้คำสั่ง `tokenizer.convert_ids_to_tokens(input_id)`

In [19]:
tokenizer.convert_ids_to_tokens(1343)

'คนร้าย'

In [20]:
[tokenizer.convert_ids_to_tokens(x) for x in tokenized_input["input_ids"] ]

['<s>', '</s>']

จะเห็นได้ว่า BERT เพิ่ม  `'▁'` เข้าไปเป็น word boundary และตัดคำบางคำออกเป็นคำย่อยๆ เช่น  `'จักรยานยนต์'` กลายเป็น  ` '▁จ',
 'ักรยานยนต์'`

ซึ่งกลายเป็นปัญหาเนื่องจาก sequence label ต้นฉบับของเรามีแค่ 9 labels  แต่ tokens จาก BERT มีถึง 20 tokens






In [21]:
len(example["conn_label"]), len(tokenized_input["input_ids"])

(0, 2)

ปัญหานี้แก้ได้โดยใช้ `word_ids` ที่ออกมาพร้อมกับ `tokenized_input`




In [22]:
print(tokenized_input.word_ids())

[None, None]


`tokenized_input.word_ids()` จะบอกเราว่า token จาก BERT มาจากคำที่เท่าไหร่ในประโยค ส่วน special token เช่น `<s>` จะ return ค่า None

เราจึงกำหนดให้ label ของ  special token เป็น -100 ซึ่งเป็นค่าที่จะถูก ignore ในขั้นตอน training
และกำหนดให้ subword ย่อย ทุกอันที่มาจากคำเดียวกัน มี label เดียวกัน

In [23]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["conn_label"][i] for i in word_ids]
aligned_labels

[-100, -100]

In [24]:
len(aligned_labels), len(tokenized_input["input_ids"])

(2, 2)

วิธีการ map label นั้นมีสองแบบคือ


1.   Label ทุก subword ให้เหมือนกัน
2.   Label เฉพาะ subword token แรก และให้ subword ที่เหลือ เป็น -100

โดยในตัวอย่างนี้เราจะใช้วิธีแรก



In [25]:
label_all_tokens = True
#True: all tokens have same label as original word
#False: only the first token have same label as original word

In [26]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    tokens = [tokenizer.convert_ids_to_tokens(x) for x in tokenized_inputs["input_ids"] ]
    ids=[]
    labels = []
    for i, label in enumerate(examples["conn_label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # Set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        ids.append(word_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = ids
    tokenized_inputs["tokens_bert"] = tokens
    return tokenized_inputs


In [27]:
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True)

## Fine-tuning the model

In [28]:
label_list = data['train'].features["conn_label"].feature.names

In [29]:
label_list

['CON', 'N']

In [35]:
model = AutoModelForTokenClassification.from_pretrained(lm_name, num_labels=len(label_list))
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"discon model/'{model_name}-{num_epoch} epoch-{condition}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epoch,
    save_strategy=save,
    save_total_limit = 2,
    remove_unused_columns=False,
    #load_best_model_at_end=True,
    weight_decay=0.01,
    push_to_hub=False,
)
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load("seqeval")

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['dev'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-36-b17818e3a34f>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [37]:
trainer.train()

ValueError: num_samples should be a positive integer value, but got num_samples=0

##Evaluation for gold standard Pretokenized Dataset

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets['test'])
predictions = np.argmax(predictions, axis=2)

In [ ]:
bert_prediction = [
    [(id, token, prediction) for (id, token, prediction) in zip(row_id, row_token,row_predictions) ]
    for row_id, row_token, row_predictions in zip(tokenized_datasets['test']['word_ids'],tokenized_datasets['test']['tokens_bert'],predictions)
]

gold_label = [
    [label for label in row_label ]
    for row_label in tokenized_datasets['test']['conn_label']
]

ลองดู ผล prediction จาก  BERT

In [ ]:
bert_prediction[13]

In [ ]:
len(bert_prediction[13])

เทียบกับ Label ที่ถูกต้อง

In [ ]:
gold_label[13],len(gold_label[13])

In [ ]:
print(
    [(token, label) for (token, label) in zip(tokenized_datasets['test']['tokens'][13],gold_label[13])]
    )

In [ ]:
wrap_predict = []
for row in bert_prediction:
    predict_row=[]
    previous_word=None
    for item in row:
        if(item[0] is not None):
            #skip special token
            if(item[0] !=previous_word):
                predict_row.append(item[-1])
                previous_word = item[0]
    wrap_predict.append(predict_row)

In [ ]:
print(
    [(token, label) for (token, label) in zip(tokenized_datasets['test']['tokens'][13],wrap_predict[13])]
    )

In [ ]:
def flatten(t):
    return [conn_label[item] for sublist in t for item in sublist]
flat_true_predictions = flatten(wrap_predict)
flat_true_labels = flatten(gold_label)

In [ ]:
result = classification_report(flat_true_labels,flat_true_predictions,output_dict=True,labels=conn_label)
result_df=pd.DataFrame(result).transpose()
result_df